In [9]:
from jupyter_plotly_dash import JupyterDash
import dash
import html
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import matplotlib.pyplot as plt
import base64


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# example from animal_shelter import AnimalShelter
from ModuleFourPython import AnimalShelter
#global variables for btn click tracking
newIntBtn1 = 0
newIntBtn2 = 0
newIntBtn3 = 0
newIntBtn4 = 0



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter() #made shelter object
shelter._init_(username,password)#initialized access



# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = '/home/1735746_snhu/Desktop/GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2("Jelani Lewsadder CS-340 MongoDB"))), #bold header for dash
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
                       href = "https://www.snhu.edu/" , target = "_blank" ,style={"height":"10%", "width":"10%"})),#modified image
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
   className = "row",
        style = {'display' : 'flex'},#buttons and clicks set up
            children = [
                html.Button("Water Resource",id='WR',n_clicks=0),
                html.Button("Mountain or Wilderness Rescue",id='MWR',n_clicks=0),
                html.Button("Disaster Rescue or Individual Tracking",id='DIT',n_clicks=0),
                html.Button("Reset",id='Reset',n_clicks=0)
            ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div([
    dl.Map([dl.TileLayer(), dl.LocateControl(options={'locateOptions': {'enableHighAccuracy': True}})],
           id="map-id", style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}),
]),
            
             ])])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id',"data"),#setting up buttons and clicks
    [Input('WR','n_clicks'),
    Input('MWR','n_clicks'),
    Input('DIT','n_clicks'),
    Input('Reset','n_clicks')
    ])

def on_click(btn1,btn2,btn3,btn4):#establishing globals below
    global newIntBtn1
    global newIntBtn2
    global newIntBtn3
    global newIntBtn4
    if((int(btn1) > newIntBtn1)):#the queries that register if btnx is greater than global btny
        df = pd.DataFrame.from_records(shelter.read({"$and": [ {'animal_type' : 'Dog'},{"breed":  { '$in': ["Labrador Retriever Mix","Chesapeake Bay Retriever",'Newfoundland']}},{'sex_upon_outcome' : "Intact Female"}, {"$and": [ {"age_upon_outcome_in_weeks": {"$gte": 26}}, {"age_upon_outcome_in_weeks": {"$lte": 156}}]} ]}))
        newIntBtn1 += 1
    elif((int(btn2) > newIntBtn2)):
        df = pd.DataFrame.from_records(shelter.read({"$and": [ {'animal_type' : 'Dog'},{"breed":  { '$in': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky","Rottweiler"]}},{'sex_upon_outcome' : "Intact Male"}, {"$and": [ {"age_upon_outcome_in_weeks": {"$gte": 26}}, {"age_upon_outcome_in_weeks": {"$lte": 156}}]} ]}))
        newIntBtn2 += 1
    elif((int(btn3) > newIntBtn3)):
        df = pd.DataFrame.from_records(shelter.read({"$and": [ {'animal_type' : 'Dog'},{"breed":  { '$in': ["Doberman Pinscher", "German Shepherd", "Golden Retriever","Bloodhound", "Rottweiler"]}},{'sex_upon_outcome' : "Intact Male"}, {"$and": [ {"age_upon_outcome_in_weeks": {"$gte": 20}}, {"age_upon_outcome_in_weeks": {"$lte": 300}}]} ]}))
        newIntBtn3 += 1
    elif((int(btn4) > newIntBtn4)):
        df = pd.DataFrame.from_records(shelter.read({}))
        newIntBtn4 += 1
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    return df.to_dict('records')
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode='multi',
        row_selectable='single' | 'multi',
        row_deletable=True,
        selected_rows=[],
        page_action='native',
        page_current= 0,
        page_size= 10,
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#FFD343'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) # example

    return
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[1,10]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,10])
                ])
            ])
        ])
    ]    

app